## 第9章 模型设定与数据问题

### 9.4 解释变量个数的选择

In [53]:
import pandas as pd
import statsmodels.api as sm

icecream = pd.read_stata('../2_Data/Data-2e/icecream.dta')
icecream['temp_l1'] = icecream['temp'].shift(1)# 引入temp滞后1期
icecream['temp_l2'] = icecream['temp'].shift(2)# 引入temp滞后2期

def Ols_icecream(data, X_cols:list, y_col:str,lags=0):
    X =  icecream[X_cols]
    y =  icecream[y_col]
    Model = sm.OLS(y[lags:], X[lags:])
    Results = Model.fit()
    return Results

#### 1.IC方法

回归后使用信息准则AIC和BIC，观察AIC和BIC的变化，取最小值对应的模型。

结果：
- 引入滞后1期时，IC最小，因此，解释变量中应引入滞后1期。

In [54]:
Results = Ols_icecream(icecream,
                       ['temp', 'price', 'income'], 
                       'consumption')
Results1 = Ols_icecream(icecream,
                        ['temp', 'temp_l1', 'price', 'income'], 
                        'consumption',
                        1)
Results2 = Ols_icecream(icecream,
                        ['temp', 'temp_l1', 'temp_l2', 'price', 'income'], 
                        'consumption',
                        2)

# 直接调用OLSResults的属性
print(f'【不引入temp滞后项】AIC:{Results.aic:.4f}  BIC:{Results.bic:.4f}') 
print(f'【引入temp滞后1期】AIC:{Results1.aic:.4f}  BIC:{Results1.bic:.4f}') 
print(f'【引入temp滞后2期】AIC:{Results2.aic:.4f}  BIC:{Results2.bic:.4f}') 

【不引入temp滞后项】AIC:-110.6299  BIC:-106.4263
【引入temp滞后1期】AIC:-118.0386  BIC:-112.5694
【引入temp滞后2期】AIC:-111.4727  BIC:-104.8117


#### 2. 序贯t规则

步骤：
- 设置一个最大滞后期，引入所有滞后期进行回归
- 对最大滞后期进行t检验，如不显著就向前推
- 直到找到现在的滞后期为止

结果：
1. temp_l2的p值为0.5374,不显著,不接受此最大滞后项,继续检验p-1期。
2. temp_l1的p值为0.0071,显著,接此最大滞后项。


In [55]:
Results2 = Ols_icecream(icecream,
                        ['temp', 'temp_l1', 'temp_l2', 'price', 'income'], 
                        'consumption',
                        2)

if Results2.pvalues['temp_l2']>0.05:
    print(f"temp_l2的p值为{Results2.pvalues['temp_l2']:.4f},不显著,不接受此最大滞后项,继续检验p-1期。")
else:
    print(f"temp_l2的p值为{Results2.pvalues['temp_l2']:.4f},显著,接此最大滞后项。")

temp_l2的p值为0.5374,不显著,不接受此最大滞后项,继续检验p-1期。


In [56]:
Results1 = Ols_icecream(icecream,
                        ['temp', 'temp_l1', 'price', 'income'], 
                        'consumption',
                        1)

if Results1.pvalues['temp_l1']>0.05:
    print(f"temp_l1的p值为{Results1.pvalues['temp_l1']:.4f},不显著,不接受此最大滞后项,继续检验p-1期。")
else:
    print(f"temp_l1的p值为{Results1.pvalues['temp_l1']:.4f},显著,接此最大滞后项。")

temp_l1的p值为0.0071,显著,接此最大滞后项。


### 9.5 对函数形式的检验